### Notebook to genereate binned cross sections from TC output

Assumes output is in a single netcdf file on pressure levels.

James Ruppert  
jruppert@ou.edu  
4/23/22

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt

#### Directories

In [2]:
main = "/Users/jamesruppert/code/tc_output/"
memb = !ls $main
# print(memb)

#### Time selection

In [3]:
t0 = 48
t1 = t0+24

#### Read datasets

In [10]:
datdir = main+memb[0]+'/ctl/'

# Primary bin variable
# varfil = Dataset(datdir+'dbz.nc') # this opens the netcdf file
# binvar_in = varfil.variables['dbz']
varfil = Dataset(datdir+'RTHRATLW.nc') # this opens the netcdf file
binvar_in = varfil.variables['RTHRATLW'][t0:t1,:,:,:] * 3600.*24 # K/s --> K/d
bv_shape = np.shape(binvar_in)
print("Binvar shape: ",bv_shape)
nt = bv_shape[0]
nz = bv_shape[1]

# Contour variable
varfil2 = Dataset(datdir+'W.nc') # this opens the netcdf file
binvar2_in = varfil2.variables['W'][t0:t1,:,:,:] # m/s

# Indexing variable
binfil = Dataset(datdir+'pw.nc') # this opens the netcdf file
ivar = binfil.variables['PW'][t0:t1,:,:,:]
print("PW shape: ",np.shape(ivar))

Binvar shape:  (24, 10, 740, 1400)
PW shape:  (24, 1, 740, 1400)


#### Create PW Bins

In [6]:
min=35 # mm
max=70
step=1
bins = np.arange(min,max,step)
nbins = np.size(bins)
# pw

#### Bin the target variable

In [11]:
binvar = 0
binvar = np.zeros((nbins,nt,nz)) # nbins, nt, nz
binvar2 = binvar

# for ibin in range(nbins):
for itim in range(nt):
    for ibin in range(nbins):
        indices = ((ivar[itim,0,:,:] >= bins[ibin]-0.5*step) & (ivar[itim,0,:,:] < bins[ibin]+0.5*step)).nonzero()
        tmp = binvar_in[itim,:,indices[0],indices[1]]
        binvar[ibin,itim,:] = np.mean(tmp,axis=0,dtype=np.float64)
        tmp = binvar2_in[itim,:,indices[0],indices[1]]
        binvar2[ibin,itim,:] = np.mean(tmp,axis=0,dtype=np.float64)

---
### Plotting routines

In [ ]:
# create figure
fig = plt.figure(figsize=(20,16))

# fill contour for main variable
spd = np.sqrt(u**2 + v**2)
clevs_spd = np.arange(30, 70, 10)
im = plt.contourf(lon, lat, spd, clevs_spd, cmap='GnBu', alpha=0.6, \
                 extend='max',projection=cartopy.crs.PlateCarree(), zorder=2)
cbar = plt.colorbar(im, ax=ax); cbar.ax.set_ylabel('m s$^{-1}$')

# line contours of geopotential height (in black)
g=9.81 # m/s^2
geoz = phi / g / 10 # m2/s2 --> m --> dam
clevs_phi = np.arange(800, 1000, 10) # may need to adapt these for the pressure level
im = plt.contour(lon, lat, geoz, clevs_phi, colors='black', projection=cartopy.crs.PlateCarree(), zorder=2)
ax.clabel(im, im.levels, inline=True, fontsize=13)

# additional contours (in red)
# cvar = 
# dd = ax.contour(lon, lat, cvar*1e5, np.arange(-200, 200, 10), colors='red', projection=cartopy.crs.PlateCarree(), zorder=2)
# ax.clabel(dd, dd.levels, inline=True, fontsize=13)

# wind barbs
spacing=6 #barbspacing (smaller if zoomed in)
mps_to_kts=1.94384 # conversion factor from m/s to knots for barbs
uplt = u * mps_to_kts
vplt = v * mps_to_kts
# mask for speeds < 10 kts
spd = np.sqrt(uplt**2+vplt**2)
uplt[np.where(spd < 10)] = np.nan
ax.barbs(lon[::spacing,::spacing], lat[::spacing,::spacing], uplt[::spacing,::spacing], vplt[::spacing,::spacing], zorder=2)

# add map features
ax.add_feature(cartopy.feature.LAND,facecolor="lightgray") #land color
# ax.add_feature(cartopy.feature.OCEAN) #ocean color
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.STATES)
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Zoom into selected area (comment this out to plot entire available area)
ax.set_extent(plt_area)

plt.show()